In [350]:
import requests
import pandas as pd 
import numpy as np
from datetime import datetime
import os

In [272]:
def connect_to_api():
    response  = requests.get('https://www.fotmob.com/api/transfers')
    result = response.status_code

    if result == 200:
        print('Connect to API Success')
        data = response.json()
    elif result == 404:
        print('Fail to connect API')
    return result , data



In [276]:
data = connect_to_api()[1]

Connect to API Success


In [277]:
data.keys()

dict_keys(['transfers', 'hits'])

In [283]:
def processing_data(data):
    raw_data = []
    # xử lý json format to dict -> DF
    for i in range(len(data['transfers'])):
        a =     {
            'playerId' : data['transfers'][i]['playerId'],
            
            'name' : data['transfers'][i]['name'],
            'position': data.get('transfers')[i].get('position'),
            'transferDate': data['transfers'][i]['transferDate'],
            'fromClubId': data['transfers'][i]['fromClubId'],
            'fromClub': data['transfers'][i]['fromClub'],
            'toClub': data['transfers'][i]['toClub'],
            'toClubId': data['transfers'][i]['toClubId'],
            'fromDate': data['transfers'][i]['fromDate'], 
            'toDate': data.get('transfers')[i].get('toDate'),
            'transferType': data.get('transfers')[i].get('transferType'),
            'marketValue': data.get('transfers')[i].get('marketValue'),
            'fee': data.get('transfers')[i].get('fee')
            

            

        }
        raw_data.append(a)
        # break
    df = pd.DataFrame(raw_data)

    df['position_name'] = df['position'].str['label']
    df['position_description'] = df['position'].str['key'].str.split('_').str[0]
    df['transferType'] = df['transferType'].str['text']
    df['fee_transfer'] = df['fee'].str['value'].replace(np.nan,0)
    df['marketValue'] = df['marketValue'].replace(np.nan,0)
    df['transferDate'] =  pd.to_datetime(df['transferDate']).dt.strftime('%d-%m-%Y')
    df['fromDate'] =  pd.to_datetime(df['fromDate']).dt.strftime('%d-%m-%Y')
    df['toDate'] =  pd.to_datetime(df['toDate']).dt.strftime('%d-%m-%Y')


    # df.head(5)
    return df




In [366]:
transfer_table = processing_data(data)
transfer_table.head()

,playerId,name,position,transferDate,fromClubId,fromClub,toClub,toClubId,fromDate,toDate,transferType,marketValue,fee,position_name,position_description,fee_transfer
0,1649302,Jayden Hibbert,None,19-06-2024,773958,Atlanta United,Birmingham Legion FC,-1,17-06-2024,31-12-2024,on loan,25000.0,"{'feeText': 'on loan', 'localizedFeeText': 'on...",None,None,0.0
1,877450,Lasse Sørensen,"{'label': 'RM', 'key': 'rightmidfielder_short'}",19-06-2024,8430,Lincoln,Huddersfield,9796,01-07-2024,30-06-2027,contract,550000.0,"{'feeText': 'fee', 'localizedFeeText': 'transf...",RM,rightmidfielder,470000.0
2,607331,Alistair Coote,"{'label': 'LW', 'key': 'leftwinger_short'}",19-06-2024,2,Free agent,Shelbourne,5751,01-07-2024,NaN,contract,130000.0,"{'feeText': 'free transfer', 'localizedFeeText...",LW,leftwinger,0.0
3,568581,Alex Bass,"{'label': 'GK', 'key': 'keeper'}",19-06-2024,8472,Sunderland,Notts County,9819,01-07-2024,30-06-2027,contract,240000.0,None,GK,keeper,0.0
4,832826,Sam Bone,"{'label': 'CB', 'key': 'centerback_short'}",19-06-2024,8131,Maidstone United,Shelbourne,5751,01-07-2024,30-11-2024,contract,110000.0,"{'feeText': 'free transfer', 'localizedFeeText...",CB,centerback,0.0


In [367]:
date_transfer = list(transfer_table['transferDate'].unique())
date_transfer

['19-06-2024',
 '18-06-2024',
 '17-06-2024',
 '16-06-2024',
 '15-06-2024',
 '14-06-2024']

In [378]:
for i in date_transfer:
    file = transfer_table[transfer_table['transferDate'] == i]
    date = datetime.strptime(i,"%d-%m-%Y")
    year = date.year
    month = date.month
    day = date.day
    
    path = f'data/transfer/{year}/{month}'
    file_path = f'{path}/{i}'
    # Create directory if not exists 
    os.makedirs(path,exist_ok=True)
    
    # Write to directory
    file.to_csv(file_path, index=False)
    print(file_path)
    # break

data/transfer/2024/6
data/transfer/2024/6
data/transfer/2024/6
data/transfer/2024/6
data/transfer/2024/6
data/transfer/2024/6
